# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [26]:
import logging
import os 
import csv 

import numpy as np 
import pandas as pd 
import pkg_resources
from matplotlib import pyplot as plt 
from sklearn.model_selection import train_test_split

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.44.0


## Dataset

### Overview
The dataset I will be using for this project is the Top Hits Spotify from 2000-2019 dataset taken from kaggle. The goal of this project will be to predict song popularity given the available features. Popularity is measured on a continuous scale, thus this will be a **regression** problem. 


The code below will download the dataset from my github repository and create a data asset. 

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-capstone'

experiment=Experiment(ws, experiment_name)

In [3]:
found = False
key = "spotify-songs-dataset"
description_text = "Top Hits Spotify from 2000-2019"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        data = 'https://raw.githubusercontent.com/ash-mohan/azureMLCapstone/main/starter_file/data/songs_normalize.csv'
        dataset = Dataset.Tabular.from_delimited_files(data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,duration_ms,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,228748.124500,2009.49400,59.872500,0.667437,0.720366,5.378000,-5.512435,0.553500,0.103568,0.128955,0.015226,0.181216,0.551690,120.122558
std,39136.569008,5.85996,21.335577,0.140416,0.152745,3.615059,1.933482,0.497254,0.096159,0.173346,0.087771,0.140669,0.220864,26.967112
min,113000.000000,1998.00000,0.000000,0.129000,0.054900,0.000000,-20.514000,0.000000,0.023200,0.000019,0.000000,0.021500,0.038100,60.019000
25%,203580.000000,2004.00000,56.000000,0.581000,0.622000,2.000000,-6.490250,0.000000,0.039600,0.014000,0.000000,0.088100,0.386750,98.985750
50%,223279.500000,2010.00000,65.500000,0.676000,0.736000,6.000000,-5.285000,1.000000,0.059850,0.055700,0.000000,0.124000,0.557500,120.021500
75%,248133.000000,2015.00000,73.000000,0.764000,0.839000,8.000000,-4.167750,1.000000,0.129000,0.176250,0.000068,0.241000,0.730000,134.265500
max,484146.000000,2020.00000,89.000000,0.975000,0.999000,11.000000,-0.276000,1.000000,0.576000,0.976000,0.985000,0.853000,0.973000,210.851000


In [12]:
df.info()

print(f"\n\nRows: {df.shape[0]} \nColumns: {df.shape[1]}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist            2000 non-null   object 
 1   song              2000 non-null   object 
 2   duration_ms       2000 non-null   int64  
 3   explicit          2000 non-null   bool   
 4   year              2000 non-null   int64  
 5   popularity        2000 non-null   int64  
 6   danceability      2000 non-null   float64
 7   energy            2000 non-null   float64
 8   key               2000 non-null   int64  
 9   loudness          2000 non-null   float64
 10  mode              2000 non-null   int64  
 11  speechiness       2000 non-null   float64
 12  acousticness      2000 non-null   float64
 13  instrumentalness  2000 non-null   float64
 14  liveness          2000 non-null   float64
 15  valence           2000 non-null   float64
 16  tempo             2000 non-null   float64


We can see that there are no missing values , so we will not have to drop nulls. However, this data is meant to be from 2000-2019 and we can see from the output above there are some outliers outside of this range. We will drop these rows along with any duplicate rows. 

In [18]:
new_df = df[df['year'].between(2000, 2019)].drop_duplicates()
new_df.describe()

,duration_ms,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,1899.000000,1899.000000,1899.000000,1899.000000,1899.000000,1899.000000,1899.000000,1899.000000,1899.000000,1899.000000,1899.000000,1899.000000,1899.000000,1899.000000
mean,228123.525540,2009.720379,59.558715,0.667637,0.721256,5.379147,-5.503538,0.553976,0.104413,0.127722,0.015525,0.181934,0.552579,120.115425
std,39116.687604,5.725938,21.683909,0.140547,0.152511,3.610882,1.931067,0.497209,0.096523,0.172951,0.089036,0.140827,0.220361,26.986107
min,113000.000000,2000.000000,0.000000,0.129000,0.054900,0.000000,-20.514000,0.000000,0.023200,0.000019,0.000000,0.021500,0.038100,60.019000
25%,203273.000000,2005.000000,56.000000,0.581000,0.624500,2.000000,-6.480000,0.000000,0.039900,0.013550,0.000000,0.088750,0.389500,98.605000
50%,222920.000000,2010.000000,65.000000,0.676000,0.738000,6.000000,-5.279000,1.000000,0.061300,0.055300,0.000000,0.125000,0.559000,120.028000
75%,247086.000000,2015.000000,73.000000,0.765000,0.839000,8.000000,-4.167000,1.000000,0.130500,0.175000,0.000064,0.242000,0.730500,134.124500
max,484146.000000,2019.000000,89.000000,0.975000,0.999000,11.000000,-0.276000,1.000000,0.576000,0.976000,0.985000,0.853000,0.973000,210.851000


In [20]:
print(f"Rows: {new_df.shape[0]} \nColumns: {new_df.shape[1]}")

Rows: 1899 
Columns: 18


In [25]:
# only 2000 data points so test size will be small 
train_data, test_data = train_test_split(df, test_size=0.1, random_state=42)

print(f"Training Data: {train_data.shape[0]} entries")
print(f"Testing Data: {test_data.shape[0]} entries")

Training Data: 1800 entries
Testing Data: 200 entries


In [29]:
# save csv file and upload to datastore
pd.DataFrame(train_data).to_csv("data/train_dataset.csv", index=False)
pd.DataFrame(test_data).to_csv("data/test_dataset.csv", index=False)
datastore = ws.get_default_datastore()
datastore.upload(src_dir='./data', target_path='spotify_exp', overwrite=True, show_progress=True)

 # datastore for AutoML
training_data = TabularDatasetFactory.from_delimited_files(path=datastore.path('spotify_exp/train_dataset.csv'))
testing_data = TabularDatasetFactory.from_delimited_files(path=datastore.path('spotify_exp/test_dataset.csv'))


Uploading an estimated of 5 files
Uploading ./data/.amlignore
Uploaded ./data/.amlignore, 1 files out of an estimated total of 5
Uploading ./data/.amlignore.amltmp
Uploaded ./data/.amlignore.amltmp, 2 files out of an estimated total of 5
Uploading ./data/songs_normalize.csv
Uploaded ./data/songs_normalize.csv, 3 files out of an estimated total of 5
Uploading ./data/test_dataset.csv
Uploaded ./data/test_dataset.csv, 4 files out of an estimated total of 5
Uploading ./data/train_dataset.csv
Uploaded ./data/train_dataset.csv, 5 files out of an estimated total of 5
Uploaded 5 files
Uploading an estimated of 5 files
Uploading ./data/.amlignore
Uploaded ./data/.amlignore, 1 files out of an estimated total of 5
Uploading ./data/.amlignore.amltmp
Uploaded ./data/.amlignore.amltmp, 2 files out of an estimated total of 5
Uploading ./data/songs_normalize.csv
Uploaded ./data/songs_normalize.csv, 3 files out of an estimated total of 5
Uploading ./data/test_dataset.csv
Uploaded ./data/test_dataset.cs

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [22]:
from azureml.train.automl.utilities import get_primary_metrics

get_primary_metrics("regression")

['normalized_root_mean_squared_error',
 'spearman_correlation',
 'r2_score',
 'normalized_mean_absolute_error']

In [32]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 3)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded......................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [34]:
automl_settings = {
    "n_cross_validations": 5,
    "iteration_timeout_minutes": 10,
    "experiment_timeout_hours": .25,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'normalized_root_mean_squared_error'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "regression",
                             training_data=training_data,
                             label_column_name="popularity",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [35]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-capstone,AutoML_31be47e9-b5e6-4b5c-9480-10745c1b679f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [36]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
